# Práctico 2: Recomendación de videojuegos

Autor: Mariano Zibecchi  
mzibecchi@gmail.com . 
  
[Recomendacion basada en usuarios](#Sistema-de-Recomendación-Basado-en-Usuarios)
  
La resolucion de la primer parte del practico, consiste en la definicion de una funcion __recomend_games_for_user__

La funcion recibe como parametros el username, el modelo entrenado y la cantidad de recomendaciones que se desea recibir.

Retorna como resultado una lista de juegos recomendados.
  
[Recomendacion por contenido](#Sistema-de-Recomendación-Basado-en-Juegos)  

Para la segunda parte del practica, desarrollamos la funcion __recomend_similar_games__

La funcion recibe como parametros un nombre de un juego.

Retorna como resultado una lista de juegos similares al dado como parametro.

Realizamos algunas verificaciones manuales (comparando a mano los vectores sobre los que hacemos la similitud del coseno) y observamos que las categorias de los juegos no son del todo correctas.  
  
Por ejemplo, un juego de Football, tiene en el genero los atributos Action e Indie. Un juego de NBA, tiene los atributos Adventure, Casual, Indie...

[Ver verificaciones](#Verificaciones-manuales-sobre-la-similitud-de-los-juegos)

## Obtención y limpieza del conjunto de datos

El conjunto de datos originalmente se encuentra en archivos que deberían ser de formato "JSON". Sin embargo, en realidad es un archivo donde cada línea es un objeto de JSON. Hay un problema no obstante y es que las líneas están mal formateadas, dado que no respetan el estándar JSON de utilizar comillas dobles (**"**) y en su lugar utilizan comillas simples (**'**). Afortunadamente, se pueden evaluar como diccionarios de Python, lo cuál permite trabajarlos directamente.

## Ejercicio 1: Análisis Exploratorio de Datos

Ya teniendo los datos, podemos cargarlos y empezar con el práctico. Antes que nada vamos a hacer una exploración de los datos. Lo principal a tener en cuenta para este caso es que debemos identificar las variables con las que vamos a trabajar. A diferencia del práctico anterior, este conjunto de datos no está documentado, por lo que la exploración es necesaria para poder entender que cosas van a definir nuestro sistema de recomendación.

In [1]:
import pandas as pd

### Características del conjunto de datos sobre videojuegos

Las características del conjunto de datos de videojuegos tienen la información necesaria para hacer el "vector de contenido" utilizado en el segundo sistema de recomendación. Su tarea es hacer un análisis sobre dicho conjunto de datos y descartar aquella información redundante.

In [2]:
games_df = pd.read_json("./data/steam/games.json.gz")
games_df.sample(15)

,publisher,genres,app_name,title,release_date,tags,discount_price,specs,price,early_access,id,developer,sentiment,metascore
2158,Iceberg Interactive,"[Action, Racing, Sports]",Gas Guzzlers Extreme: Full Metal Frenzy,Gas Guzzlers Extreme: Full Metal Frenzy,2014-05-12,"[Racing, Action, Sports]",NaN,"[Single-player, Multi-player, Downloadable Con...",4.99,False,277820.0,Gamepires,Positive,None
31838,"PopCap Games, Inc.",[Casual],Peggle Extreme,Peggle Extreme,2007-09-11,"[Casual, Puzzle, Free to Play, Action]",NaN,[Single-player],Third-party,False,3483.0,"PopCap Games, Inc.",Very Positive,None
9577,None,"[Indie, RPG, Strategy]",Fantasy Grounds - Winged Fiends (Token Pack),Fantasy Grounds - Winged Fiends (Token Pack),2017-04-24,"[Strategy, RPG, Indie]",NaN,"[Multi-player, Co-op, Cross-Platform Multiplay...",4.99,False,628980.0,"SmiteWorks USA, LLC",None,None
3449,None,"[Indie, RPG, Strategy]",Fantasy Grounds - D&D Warlock Class Pack,Fantasy Grounds - D&amp;D Warlock Class Pack,2015-04-07,"[RPG, Indie, Strategy, Utilities, Turn-Based, ...",NaN,"[Multi-player, Co-op, Cross-Platform Multiplay...",5.99,False,360112.0,"SmiteWorks USA, LLC",None,None
24448,None,"[Casual, Indie, RPG, Simulation, Strategy]",Tabletop Simulator - Darkest Night,Tabletop Simulator - Darkest Night,2016-01-08,"[RPG, Indie, Casual, Simulation, Strategy]",NaN,"[Single-player, Multi-player, Co-op, Shared/Sp...",6.99,False,433520.0,"Berserk Games,Victory Point Games",Positive,None
26242,None,[Action],TOUKIDEN Kiwami - Mission Collection 8-11,TOUKIDEN Kiwami - Mission Collection 8-11,2015-07-02,[Action],NaN,"[Single-player, Co-op, Downloadable Content, S...",4.99,False,380901.0,"KOEI TECMO GAMES CO., LTD.",1 user reviews,None
5221,Individual Software,[Education],CV Maker for Windows,CV Maker for Windows,2016-01-21,"[Education, Software Training, Software]",NaN,None,19.99,False,438110.0,Individual Software,1 user reviews,None
16957,None,"[Casual, Simulation]",Rocksmith® 2014 Edition – Remastered – Peter F...,Rocksmith® 2014 Edition – Remastered – Peter F...,2017-07-05,"[Casual, Simulation]",NaN,"[Single-player, Shared/Split Screen, Downloada...",2.99,False,637692.0,Ubisoft - San Francisco,1 user reviews,None
10446,Notus Games Ltd,[Casual],Travel Riddles: Trip To Italy,Travel Riddles: Trip To Italy,2017-07-13,"[Casual, Match 3, Puzzle]",NaN,"[Single-player, Steam Achievements, Steam Cloud]",4.99,False,659950.0,Notus Games Ltd,5 user reviews,None
13969,None,"[Action, Adventure, Free to Play, Indie, Strat...",Arkhan Force Grey Pack,Arkhan Force Grey Pack,2017-11-17,"[Strategy, Action, Adventure, Free to Play, In...",NaN,"[Single-player, Downloadable Content, Steam Ac...",9.99,False,756954.0,Codename Entertainment Inc.,None,None


In [3]:
len(games_df)

32135

In [4]:
games_df.dtypes

publisher          object
genres             object
app_name           object
title              object
release_date       object
tags               object
discount_price    float64
specs              object
price              object
early_access         bool
id                float64
developer          object
sentiment          object
metascore          object
dtype: object

In [5]:
games_df['id'].isnull().sum()

2

In [6]:
#games_df = games_df.dropna()
games_df[ games_df['id'].isnull() ]


,publisher,genres,app_name,title,release_date,tags,discount_price,specs,price,early_access,id,developer,sentiment,metascore
74,None,None,None,None,None,None,14.99,None,19.99,False,NaN,None,None,None
30961,"Warner Bros. Interactive Entertainment, Feral ...","[Action, Adventure]",Batman: Arkham City - Game of the Year Edition,Batman: Arkham City - Game of the Year Edition,2012-09-07,"[Action, Open World, Batman, Adventure, Stealt...",NaN,"[Single-player, Steam Achievements, Steam Trad...",19.99,False,NaN,"Rocksteady Studios,Feral Interactive (Mac)",Overwhelmingly Positive,91


In [7]:
games_df = games_df.dropna (  subset=['id'] )

In [8]:
games_df[ games_df['id'].isnull() ]

,publisher,genres,app_name,title,release_date,tags,discount_price,specs,price,early_access,id,developer,sentiment,metascore


In [9]:
games_df

,publisher,genres,app_name,title,release_date,tags,discount_price,specs,price,early_access,id,developer,sentiment,metascore
0,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",4.49,[Single-player],4.99,False,761140.0,Kotoshiro,None,None
1,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",NaN,"[Single-player, Multi-player, Online Multi-Pla...",Free To Play,False,643980.0,Secret Level SRL,Mostly Positive,None
2,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",NaN,"[Single-player, Multi-player, Online Multi-Pla...",Free to Play,False,670290.0,Poolians.com,Mostly Positive,None
3,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,弹炸人2222,2017-12-07,"[Action, Adventure, Casual]",0.83,[Single-player],0.99,False,767400.0,彼岸领域,None,None
4,None,None,Log Challenge,None,None,"[Action, Indie, Casual, Sports]",1.79,"[Single-player, Full controller support, HTC V...",2.99,False,773570.0,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32130,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,2018-01-04,"[Strategy, Indie, Casual, Simulation]",1.49,"[Single-player, Steam Achievements]",1.99,False,773640.0,"Nikita ""Ghost_RUS""",None,None
32131,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,2018-01-04,"[Strategy, Indie, Casual]",4.24,"[Single-player, Steam Achievements, Steam Clou...",4.99,False,733530.0,Sacada,None,None
32132,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,2018-01-04,"[Indie, Simulation, Racing]",1.39,"[Single-player, Steam Achievements, Steam Trad...",1.99,False,610660.0,Laush Dmitriy Sergeevich,None,None
32133,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",NaN,"[Single-player, Steam Achievements, Steam Cloud]",4.99,False,658870.0,"xropi,stev3ns",1 user reviews,None


In [10]:
games_df['id'] = games_df['id'].astype(int)

In [11]:
games_df.head()

,publisher,genres,app_name,title,release_date,tags,discount_price,specs,price,early_access,id,developer,sentiment,metascore
0,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",4.49,[Single-player],4.99,False,761140,Kotoshiro,None,None
1,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",NaN,"[Single-player, Multi-player, Online Multi-Pla...",Free To Play,False,643980,Secret Level SRL,Mostly Positive,None
2,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",NaN,"[Single-player, Multi-player, Online Multi-Pla...",Free to Play,False,670290,Poolians.com,Mostly Positive,None
3,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,弹炸人2222,2017-12-07,"[Action, Adventure, Casual]",0.83,[Single-player],0.99,False,767400,彼岸领域,None,None
4,None,None,Log Challenge,None,None,"[Action, Indie, Casual, Sports]",1.79,"[Single-player, Full controller support, HTC V...",2.99,False,773570,None,None,None


In [12]:
games_df.metascore.unique()

array([None, 96, 84, 80, 76, 70, 'NA', 69, 81, 75, 72, 66, 67, 77, 91, 89,
       83, 61, 88, 65, 94, 57, 86, 87, 92, 79, 82, 58, 74, 85, 90, 68, 71,
       60, 73, 59, 64, 54, 53, 78, 51, 44, 63, 38, 56, 49, 52, 62, 93, 48,
       34, 95, 43, 55, 24, 46, 41, 20, 39, 45, 35, 47, 40, 36, 50, 32, 37,
       33, 42, 27, 29, 30], dtype=object)

In [13]:
games_df.sentiment.unique()

array([None, 'Mostly Positive', 'Mixed', '1 user reviews',
       '3 user reviews', '8 user reviews', 'Very Positive',
       'Overwhelmingly Positive', '6 user reviews', '5 user reviews',
       '2 user reviews', 'Very Negative', 'Positive', 'Mostly Negative',
       '9 user reviews', 'Negative', '4 user reviews', '7 user reviews',
       'Overwhelmingly Negative'], dtype=object)

### Características del conjunto de datos de reviews

Este será el conjunto de datos a utilizar para obtener información sobre los usuarios y su interacción con videojuegos. Como se puede observar no hay un rating explícito, sino uno implícito a calcular, que será parte de su trabajo (deberán descubrir que característica les puede dar información que puede ser equivalente a un rating).

In [14]:
reviews_df = pd.read_json("./data/steam/reviews.json.gz")
reviews_df.sample(15)

,username,product_id,page_order,text,hours,products,date,early_access,page,compensation,found_funny,user_id
664492,TheEeveeWarrior,391540,8,...\n...\n...\nI...\nI have no words. There ar...,8.2,280.0,2015-11-21,False,5417,None,2.0,NaN
551921,goodyuppie,307690,6,It's like GTA...with kung fu....and pork buns....,48.2,4.0,2015-03-26,False,441,None,NaN,7.656120e+16
187577,Wild Salmon,356570,6,"I carried the guests, one by one, to the back ...",7.1,1309.0,2016-07-14,False,138,None,NaN,7.656120e+16
662344,Wahrheit,71250,7,BUY THIS GAME BUY THIS GAME BUY THIS GAME,7.3,1415.0,2011-03-05,False,191,None,NaN,NaN
257624,Xiphos,48700,6,10/10 IGN for best napoleonic wars community a...,709.7,50.0,2015-09-28,False,1700,None,NaN,NaN
3045,Vivian Lopez!,241600,4,I was really excited when I heard about a Cast...,46.9,362.0,2013-12-13,False,811,None,NaN,NaN
334997,Cap'n Cunt,24960,3,Arguabley the best and for certain my favourit...,40.0,76.0,2014-10-24,False,578,None,NaN,NaN
223989,Small Yachty,270370,2,hot game,16.5,14.0,2017-03-18,False,30,None,NaN,7.656120e+16
160590,adam1224,6120,9,A nicely done sidescroller brawler with reveng...,14.4,1812.0,2017-02-18,False,9,None,NaN,NaN
681630,yuGmooDehT,233740,8,Fun game to play.,14.1,1128.0,2014-01-14,False,146,None,NaN,NaN


Descripcion del Dataset:  
* hours - horas jugadas antes de emitir el review
* early_access - si se trata de una version early access del producto
**Early Access is not meant to be a form of pre-purchase, but a tool to get your game in front of Steam users and gather feedback while finishing your game. Early Access titles must deliver a playable game or usable software to the customer at the time of purchase, while pre-purchase games are delivered at a future date.**
* found_funny - otros usuarios encontraron el review divertido
* compensation - 
* products - indica los diferentes juegos q tiene la cuenta en su libreria

In [15]:
user = 'TheDoofMoments'
reviews_df[ reviews_df['username'] == user ]

,username,product_id,page_order,text,hours,products,date,early_access,page,compensation,found_funny,user_id
127491,TheDoofMoments,200210,0,I like this game,625.3,65.0,2017-07-02,False,150,None,NaN,NaN


In [16]:
user = 'SPejsMan'
reviews_df[ reviews_df['username'] == user ]

,username,product_id,page_order,text,hours,products,date,early_access,page,compensation,found_funny,user_id
0,SPejsMan,227940,0,Just one word... Balance!,23.0,92.0,2015-02-25,True,3159,None,NaN,NaN
87241,SPejsMan,282070,7,It is good to be bad.,13.8,92.0,2016-11-26,False,459,None,NaN,NaN


In [17]:
user = 'Agron Bulchard-Chataeu'
reviews_df[ reviews_df['username'] == user ]

,username,product_id,page_order,text,hours,products,date,early_access,page,compensation,found_funny,user_id


Veamos un juego en particular __Total War™: ROME II - Emperor Edition__  
y los reviews que tuvo...  


In [18]:
games_df[ games_df ['id'] == 214950 ]

,publisher,genres,app_name,title,release_date,tags,discount_price,specs,price,early_access,id,developer,sentiment,metascore
30024,SEGA,[Strategy],Total War™: ROME II - Emperor Edition,Total War™: ROME II - Emperor Edition,2013-09-02,"[Strategy, Historical, Turn-Based Strategy, Gr...",NaN,"[Single-player, Multi-player, Steam Trading Ca...",59.95,False,214950,Creative Assembly,Mostly Positive,76


In [19]:
roma_reviews_df = reviews_df [ reviews_df['product_id'] == 214950]
roma_reviews_df

,username,product_id,page_order,text,hours,products,date,early_access,page,compensation,found_funny,user_id
7,CaptainPlanet,214950,1,"If you like slaughtering in the name of Rome, ...",203.3,274.0,2016-10-27,False,285,None,NaN,7.656120e+16
1449,Ciddie Fobbler,214950,5,Does ok at the things that are ok,130.6,115.0,2015-08-08,False,621,None,NaN,NaN
1896,turtleownage,214950,1,"Super fun, battles are epic and a treat to wat...",7.1,40.0,2017-10-25,False,78,None,NaN,7.656120e+16
2259,battlegalactika,214950,8,The combat is simply broken\nIt doesnt feel li...,76.9,5.0,2017-05-12,False,153,None,NaN,7.656120e+16
2468,Terminally Chill™,214950,8,Stiltzkin: are you high\nStiltzkin: youve been...,43.0,17.0,2015-02-15,False,816,None,10.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
697932,sosacharly30,214950,1,WORSE♥♥♥♥♥♥I HAVE EVER PLAYED BY CA I SWEAR SL...,805.1,23.0,2014-08-01,False,1195,None,NaN,7.656120e+16
698520,DrFDisK,214950,1,Worse than I expected. Still not a finished ga...,6.1,265.0,2015-06-28,False,664,None,NaN,NaN
699106,AMP2533,214950,1,I personally like this game but after having s...,96.4,95.0,2016-02-10,False,467,None,NaN,NaN
699216,Adjusted for Infeeltion,214950,4,A great strategy game built on the reliable to...,70.8,170.0,2016-11-23,False,266,None,NaN,NaN


In [20]:
user = 'sosacharly30'
reviews_df[ reviews_df['username'] == user ].text

697932    WORSE♥♥♥♥♥♥I HAVE EVER PLAYED BY CA I SWEAR SL...
Name: text, dtype: object

In [21]:
roma_reviews_df.username.value_counts()

Gutenisse                  2
Analf Shitler              2
Sitout.AmnesiacGaming      2
bart.es                    2
valgrindkennels            2
                          ..
[IG] Tomamellor [Max]      1
[LOST] Spedsnaz            1
Corrino                    1
Adjusted for Infeeltion    1
♦TUCK3R♦                   1
Name: username, Length: 1228, dtype: int64

In [22]:
user = 'Caesar'
roma_reviews_df[ roma_reviews_df['username'] == user ]

,username,product_id,page_order,text,hours,products,date,early_access,page,compensation,found_funny,user_id
209383,Caesar,214950,4,can't compare to rome 1,686.2,36.0,2014-09-29,False,1081,None,NaN,NaN
647322,Caesar,214950,4,I love everything simply because of the mod ba...,495.3,86.0,2014-09-20,False,1117,None,NaN,NaN


In [23]:
len(reviews_df)

700000

In [24]:
reviews_df.describe()

,product_id,page_order,hours,products,page,found_funny,user_id
count,700000.000000,700000.000000,697558.000000,698708.000000,700000.000000,107375.000000,2.855870e+05
mean,251130.840463,4.485711,111.498797,236.192551,890.908570,7.738217,7.656120e+16
std,150044.054746,2.875279,385.359458,485.337170,1923.769739,71.931147,1.009730e+08
min,10.000000,0.000000,0.000000,1.000000,1.000000,1.000000,7.656120e+16
25%,203770.000000,2.000000,4.000000,45.000000,52.000000,1.000000,7.656120e+16
50%,252490.000000,4.000000,15.200000,110.000000,237.000000,1.000000,7.656120e+16
75%,346110.000000,7.000000,59.300000,246.000000,828.000000,2.000000,7.656120e+16
max,773900.000000,9.000000,18570.900000,12832.000000,18371.000000,6956.000000,7.656120e+16


In [25]:
reviews_df.dtypes

username                object
product_id               int64
page_order               int64
text                    object
hours                  float64
products               float64
date            datetime64[ns]
early_access              bool
page                     int64
compensation            object
found_funny            float64
user_id                float64
dtype: object

In [26]:
reviews_df.compensation.value_counts()

Product received for free    13286
Name: compensation, dtype: int64

#### Juegos con mas reviews

In [27]:
reviews_df.groupby('product_id').agg({'page_order':'count'}).sort_values('page_order', ascending=False).head()

,page_order
product_id,
440,16570
252490,9161
49520,6349
377160,6335
271590,5239


In [28]:
reviews_df.found_funny.value_counts()

1.0       62816
2.0       17737
3.0        7511
4.0        4259
5.0        2585
          ...  
285.0         1
1153.0        1
1170.0        1
432.0         1
253.0         1
Name: found_funny, Length: 589, dtype: int64

# Sistema de Recomendación Basado en Usuarios

Este sistema de recomendación deberá entrenar un algoritmo y desarrollar una interfaz que, dado un usuario, le devuelva una lista con los juegos más recomendados.

In [29]:
#cant_horas_user = reviews_df.groupby(['username']).agg( cant_horas = pd.NamedAgg(column='hours', aggfunc=sum))
cant_horas_user = reviews_df.groupby(['username']).hours.sum()
cant_horas_user

username
!                      123.3
!             *          5.6
! 5tryx                  0.9
! DeadlyDeal !           6.3
! Indelible             58.3
                       ...  
󰀈Spectra󰀈                4.9
󰀍 loopuleasa 󰀍          96.9
󰀓Oxymoronicphalanx󰀓      0.4
󰀕СЭНС󰀕                  21.1
󰀗 Lolicage               7.5
Name: hours, Length: 495383, dtype: float64

In [30]:
reviews_df['cant_horas_user'] = reviews_df['username'].apply(lambda x: cant_horas_user[x] 
                                                          if x in cant_horas_user else -1)


In [31]:
reviews_df[ reviews_df['username'] == 'Caesar' ]


,username,product_id,page_order,text,hours,products,date,early_access,page,compensation,found_funny,user_id,cant_horas_user
77242,Caesar,221100,5,"Forever ""Early Access"".",67.0,21.0,2016-02-01,True,2517,None,NaN,7.656120e+16,1723.3
195973,Caesar,327070,0,"Game's unplayable (I know it's EA, but still) ...",1.3,280.0,2016-07-08,True,73,None,NaN,NaN,1723.3
209383,Caesar,214950,4,can't compare to rome 1,686.2,36.0,2014-09-29,False,1081,None,NaN,NaN,1723.3
235587,Caesar,219640,8,"Total loss of money,time and resources.",5.1,88.0,2017-03-27,False,290,None,NaN,7.656120e+16,1723.3
275411,Caesar,204100,0,Good pc port I can max settings @ 1080p with 2...,9.5,48.0,2015-01-31,False,420,None,NaN,NaN,1723.3
285946,Caesar,441790,0,For an EA this game is fun. Graphics are amazi...,81.8,37.0,2016-04-30,True,46,None,NaN,NaN,1723.3
298652,Caesar,588430,7,pay to win.,3.3,76.0,2017-06-20,False,249,None,NaN,NaN,1723.3
349061,Caesar,223850,3,The ultimate pay to win game.\n10/10,19.3,183.0,2017-04-06,False,18,None,3.0,NaN,1723.3
397510,Caesar,582660,8,Incredible design and textures. Game has great...,120.5,92.0,2017-07-04,False,346,None,NaN,7.656120e+16,1723.3
408255,Caesar,204300,8,Awesomenauts is a side-scrolling 3v3 DOTA-styl...,100.3,76.0,2015-05-14,False,545,None,NaN,NaN,1723.3


In [32]:
reviews_df['rating'] = reviews_df['hours'] / reviews_df['cant_horas_user']


In [33]:
reviews_df[ reviews_df['username'] == 'Caesar' ]


,username,product_id,page_order,text,hours,products,date,early_access,page,compensation,found_funny,user_id,cant_horas_user,rating
77242,Caesar,221100,5,"Forever ""Early Access"".",67.0,21.0,2016-02-01,True,2517,None,NaN,7.656120e+16,1723.3,0.038879
195973,Caesar,327070,0,"Game's unplayable (I know it's EA, but still) ...",1.3,280.0,2016-07-08,True,73,None,NaN,NaN,1723.3,0.000754
209383,Caesar,214950,4,can't compare to rome 1,686.2,36.0,2014-09-29,False,1081,None,NaN,NaN,1723.3,0.398190
235587,Caesar,219640,8,"Total loss of money,time and resources.",5.1,88.0,2017-03-27,False,290,None,NaN,7.656120e+16,1723.3,0.002959
275411,Caesar,204100,0,Good pc port I can max settings @ 1080p with 2...,9.5,48.0,2015-01-31,False,420,None,NaN,NaN,1723.3,0.005513
285946,Caesar,441790,0,For an EA this game is fun. Graphics are amazi...,81.8,37.0,2016-04-30,True,46,None,NaN,NaN,1723.3,0.047467
298652,Caesar,588430,7,pay to win.,3.3,76.0,2017-06-20,False,249,None,NaN,NaN,1723.3,0.001915
349061,Caesar,223850,3,The ultimate pay to win game.\n10/10,19.3,183.0,2017-04-06,False,18,None,3.0,NaN,1723.3,0.011199
397510,Caesar,582660,8,Incredible design and textures. Game has great...,120.5,92.0,2017-07-04,False,346,None,NaN,7.656120e+16,1723.3,0.069924
408255,Caesar,204300,8,Awesomenauts is a side-scrolling 3v3 DOTA-styl...,100.3,76.0,2015-05-14,False,545,None,NaN,NaN,1723.3,0.058202


Verificamos que no hayan quedado valores nulos en el rating que calculamos

In [34]:
reviews_df['rating'].isnull().sum()

2616

Vamos a reemplazar los nulos por 0

In [35]:
import numpy as np

reviews_df['rating'] = reviews_df['rating'].replace(to_replace = np.nan, value =0 )

In [36]:
reviews_df['rating'].isnull().sum()

0

In [37]:
import pandas as pd
import seaborn as sns

ax = sns.distplot(reviews_df['rating'])

### Armamos la matriz de usuario-contenido

In [38]:
muc_df = reviews_df[ ['username','product_id','rating'] ]
muc_df.head()

,username,product_id,rating
0,SPejsMan,227940,0.625000
1,Spodermen,270170,0.308176
2,josh,41700,0.052637
3,Sammyrism,332310,0.054054
4,moonmirroir,303210,1.000000


In [39]:
len(reviews_df)

700000

In [40]:
len(muc_df)

700000

In [41]:
muc_df = muc_df.sample(50000)

In [42]:
muc_df.head()

,username,product_id,rating
109607,Dostoyevsky,346110,1.000000
470547,fallmoonight,32370,1.000000
206216,GreybeardTheMagnificent,371330,1.000000
137847,Masanori,620,1.000000
436638,Rabbi Vikerstein,256290,0.717868


In [43]:
from surprise import Dataset, Reader, KNNWithMeans, BaselineOnly
from surprise.accuracy import rmse
from surprise.model_selection import cross_validate, train_test_split


In [44]:
#ratings_train, ratings_test = train_test_split(ratings, test_size=0.2)

# model = KNNWithMeans(k=3).fit( ratings.build_full_trainset() )

reader = Reader(rating_scale=(muc_df.rating.min(), muc_df.rating.max()))

ratings = Dataset.load_from_df( muc_df[["username", "product_id", "rating"]], reader)

trainset = ratings.build_full_trainset()
                               
bsl_options = {'method': 'als',
               'n_epochs': 5,
               'reg_u': 12,
               'reg_i': 5
               }
model = BaselineOnly(bsl_options=bsl_options)

model.fit( trainset )


Estimating biases using als...


### Trabajamos en la funcion de recomendacion

In [45]:
games = games_df['id'].unique()

In [46]:
def get_game_name( game_id ):
    return games_df [ games_df['id'] == game_id ]['app_name'].tolist()


El desarrollo de la funcion esta inspirado en el siguiente trabajo:  
https://blog.cambridgespark.com/tutorial-practical-introduction-to-recommender-systems-dbe22848392b


In [47]:
def recomend_games_for_user( user, model, cant_games  ):
    
    games_rated_by_user = []
    games_to_predict_for_user = []
    testset = []
    
    # armamos la lista de los juegos que el usuario rankeo

    games_rated_by_user = muc_df.loc[ muc_df['username'] ==  user, 'product_id']
    
    # calculamos la lista de juegos que tenemos que predecir (i.e. todos - los que el usuario rankeo)
    
    games_to_predict_for_user = np.setdiff1d(games, games_rated_by_user)

    testset = [[user, prod_id, 4.] for prod_id in games_to_predict_for_user]
    
    preds = model.test(testset)

    df = pd.DataFrame(preds, columns=['uid', 'iid', 'rui', 'est', 'details'])

    best_predictions = df.sort_values(by='est', ascending=False)[:cant_games]
    
    rec_games = []
    for game_id in best_predictions.iid:
        rec_games.append( get_game_name(game_id ))
        
    return rec_games, best_predictions


In [48]:
recs = []
recs, best = recomend_games_for_user( 'Lyra', model, 5 )
for r in recs:
    print(r)
best

['Football Manager 2017']
['Counter-Strike: Global Offensive']
['Stronghold Kingdoms']
['FINAL FANTASY XIV Online']
['Tree of Savior (English Ver.)']


,uid,iid,rui,est,details
17570,Lyra,482730,4.0,0.862664,{'was_impossible': False}
25,Lyra,730,4.0,0.854263,{'was_impossible': False}
1240,Lyra,47410,4.0,0.853565,{'was_impossible': False}
1027,Lyra,39210,4.0,0.853290,{'was_impossible': False}
10930,Lyra,372000,4.0,0.848386,{'was_impossible': False}


In [49]:
recs = []
recs, best = recomend_games_for_user( 'Codes', model, 5 )
for r in recs:
    print(r)
best

['Football Manager 2017']
['Counter-Strike: Global Offensive']
['Stronghold Kingdoms']
['FINAL FANTASY XIV Online']
['Tree of Savior (English Ver.)']


,uid,iid,rui,est,details
17571,Codes,482730,4.0,0.919197,{'was_impossible': False}
25,Codes,730,4.0,0.910796,{'was_impossible': False}
1240,Codes,47410,4.0,0.910098,{'was_impossible': False}
1027,Codes,39210,4.0,0.909823,{'was_impossible': False}
10930,Codes,372000,4.0,0.904919,{'was_impossible': False}


In [50]:
recs = []
recs, best = recomend_games_for_user( 'Sine', model, 5 )
for r in recs:
    print(r)
best

['Football Manager 2017']
['Counter-Strike: Global Offensive']
['Stronghold Kingdoms']
['FINAL FANTASY XIV Online']
['Tree of Savior (English Ver.)']


,uid,iid,rui,est,details
17570,Sine,482730,4.0,0.885297,{'was_impossible': False}
25,Sine,730,4.0,0.876896,{'was_impossible': False}
1240,Sine,47410,4.0,0.876198,{'was_impossible': False}
1027,Sine,39210,4.0,0.875923,{'was_impossible': False}
10929,Sine,372000,4.0,0.871019,{'was_impossible': False}


# Sistema de Recomendación Basado en Juegos

Similar al caso anterior, con la diferencia de que este sistema espera como entrada el nombre de un juego y devuelve una lista de juegos similares. El sistema deberá estar programado en base a información de contenido de los juegos (i.e. filtrado basado en contenido o sistema híbrido).

In [129]:
games_df.sample(15)

,publisher,genres,app_name,title,release_date,tags,discount_price,specs,price,early_access,...,Early Access,Audio Production,Racing,Education,Photo Editing,Indie,RPG,Animation &amp; Modeling,Action,Casual
1315,Ubisoft,"[Action, Racing]",Trials Evolution: Gold Edition,Trials Evolution: Gold Edition,2013-03-21,"[Racing, Action, Multiplayer, Level Editor, Pl...",NaN,"[Single-player, Multi-player, Partial Controll...",19.99,False,...,0,0,1,0,0,0,0,0,1,0
2140,Strategy First,[Strategy],Warrior Kings: Battles,Warrior Kings: Battles,2003-09-30,"[Strategy, RTS, Fantasy]",NaN,[Single-player],9.99,False,...,0,0,0,0,0,0,0,0,0,0
30594,Ubisoft,[Action],Tom Clancy's Ghost Recon Future Soldier - Arct...,Tom Clancy's Ghost Recon Future Soldier - Arct...,2012-10-25,[Action],NaN,"[Single-player, Multi-player, Co-op, Downloada...",9.99,False,...,0,0,0,0,0,0,0,0,1,0
28133,Netcore Games,"[Adventure, Indie, RPG]",Tales of Maj'Eyal - Ashes of Urh'Rok,Tales of Maj'Eyal - Ashes of Urh'Rok,2014-10-27,"[RPG, Indie, Adventure, Rogue-like, Turn-Based...",NaN,"[Single-player, Downloadable Content, Steam Ac...",3.99,False,...,0,0,0,0,0,1,1,0,0,0
18904,SEGA,"[RPG, Strategy]",Endless Legend™ - Forgotten Love Add-on,Endless Legend™ - Forgotten Love Add-on,2017-03-02,"[Strategy, RPG]",NaN,"[Single-player, Multi-player, Online Multi-Pla...",Free,False,...,0,0,0,0,0,0,1,0,0,0
26916,None,"[Action, Adventure, Free to Play, Indie, Simul...",Void Destroyer - Mini Sandbox,Void Destroyer - Mini Sandbox,2015-04-28,"[Action, Adventure, Free to Play, Indie, Simul...",NaN,"[Single-player, Downloadable Content, Partial ...",None,False,...,0,0,0,0,0,1,0,0,1,0
8225,Gestmorph Games,"[Adventure, Free to Play, RPG]",ERR - 001,ERR - 001,2016-12-13,"[Free to Play, Adventure, RPG, RPGMaker]",NaN,"[Single-player, Steam Trading Cards]",Free To Play,False,...,0,0,0,0,0,0,1,0,0,0
15299,NedoStudio,"[Casual, Indie]",Ablepsia,Ablepsia,2017-09-27,"[Indie, Casual, 2D, Point & Click]",NaN,"[Single-player, Steam Achievements]",0.99,False,...,0,0,0,0,0,1,0,0,0,1
3069,Grey Box,[Strategy],Grey Goo,Grey Goo,2015-01-23,"[Strategy, RTS, Sci-fi, Multiplayer, Great Sou...",NaN,"[Single-player, Multi-player, Steam Achievemen...",29.99,False,...,0,0,0,0,0,0,0,0,0,0
11445,Alakhaine Specialist Consulting Limited,"[Casual, Indie]",Mr Rabbit's Jigsaw Puzzle,Mr Rabbit's Jigsaw Puzzle,2017-09-13,"[Indie, Casual]",NaN,"[Single-player, Stats]",1.99,False,...,0,0,0,0,0,1,0,0,0,1


In [71]:
games_df['app_name'].isnull().sum()

1

### Trabajamos sobre los generos de los juegos

In [72]:
# 66 games_df['genres'] = games_df['genres'].fillna('', inplace=True)
games_df['genres'].isnull().sum()

0

In [73]:
#games_df['genres'] = games_df['genres'].replace(np.nan, ['No Genre'])
games_df.dropna( how='any', subset=['genres','app_name'], inplace=True )

In [74]:
games_df['genres']

0            [Action, Casual, Indie, Simulation, Strategy]
1                     [Free to Play, Indie, RPG, Strategy]
2        [Casual, Free to Play, Indie, Simulation, Sports]
3                              [Action, Adventure, Casual]
5                          [Action, Adventure, Simulation]
                               ...                        
32129                   [Action, Adventure, Casual, Indie]
32130                [Casual, Indie, Simulation, Strategy]
32131                            [Casual, Indie, Strategy]
32132                          [Indie, Racing, Simulation]
32133                                      [Casual, Indie]
Name: genres, Length: 28850, dtype: object

In [75]:
games_df['genres'].isnull().sum()

0

In [76]:
#games_df['genres'] = games_df['genres'].apply(lambda x: x.split(","))
#for genre in games_df['genres']:
#    #print(genre)
#    for g in genre:
#        print (g)

In [77]:
genres = set(g for G in games_df['genres'] for g in G)

for g in genres:
    games_df[g] = games_df.genres.transform(lambda x: int(g in x))
    
games_genres = games_df.drop(columns=['id', 'publisher','genres', 'app_name', 'title', 'release_date', 
                                      'tags', 'discount_price', 'specs', 'price', 'early_access', 
                                     'developer', 'sentiment', 'metascore'])

In [78]:
genres

{'Accounting',
 'Action',
 'Adventure',
 'Animation &amp; Modeling',
 'Audio Production',
 'Casual',
 'Design &amp; Illustration',
 'Early Access',
 'Education',
 'Free to Play',
 'Indie',
 'Massively Multiplayer',
 'Photo Editing',
 'RPG',
 'Racing',
 'Simulation',
 'Software Training',
 'Sports',
 'Strategy',
 'Utilities',
 'Video Production',
 'Web Publishing'}

In [64]:
games_genres.head()

,Adventure,Strategy,Design &amp; Illustration,Free to Play,Web Publishing,Massively Multiplayer,Accounting,Sports,Simulation,Utilities,...,Early Access,Audio Production,Racing,Education,Photo Editing,Indie,RPG,Animation &amp; Modeling,Action,Casual
0,0,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,1,1
1,0,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,1,1,0,0,0
2,0,0,0,1,0,0,0,1,1,0,...,0,0,0,0,0,1,0,0,0,1
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
5,1,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0


In [79]:
def game_finder( title ):
    
    return games_df[ games_df['app_name'].str.contains(title)]['app_name'].tolist()


In [141]:
 game_finder('NBA')

['NBA 2K17',
 'NBA Playgrounds - Hot ’N Frosty',
 'NBA 2K18',
 'NBA Playgrounds - Unlock Roster',
 'NBA Playgrounds',
 'NBA 2K16']

In [139]:
 game_finder('Football')

['Lords of Football',
 'Lords of Football: Super Training',
 'Lords of Football: Eastern Europe',
 'Lords of Football: United States',
 'Front Page Sports Football',
 'Football Mogul 2014',
 'Football Tactics',
 'Football Tactics Demo',
 'Super Arcade Football',
 'Football Mogul 15',
 'Football Manager Touch 2017 - No Transfer Windows',
 'Football Manager Touch 2017 - Son Generated',
 'Football Manager 2017',
 'Football Manager Touch 2017 - Unlimited Scouting',
 'Football Manager Touch 2017',
 'FreeStyleFootball - Starter Pack',
 'Table Football Pro',
 'Canadian Football 2017',
 'Axis Football 2017',
 'Hyper Knights - Minion Football',
 'Australian Football Coach',
 'Football Manager Touch 2018 - Son Generated',
 'Football Manager 2018 - In-Game Editor',
 'Football Manager Touch 2018 - Starting from the Bottom Challenge',
 'Football Manager Touch 2018 - Unlimited Scouting',
 'Pro Strategy Football 2018',
 'Football Manager 2018',
 'Football Manager Touch 2018',
 'Football Manager Touch

In [84]:
len( game_finder('Ironbound'))

1

In [121]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_sim = cosine_similarity(games_genres, games_genres)

games_idx = dict(zip( games_df['app_name'], list(games_df.index)))


In [132]:
def recomend_similar_games( game_title ):
 

    title = game_finder( game_title )[0]

    n_recommendations = 5

    idx = games_idx[title]
    print('idx game = ', idx )
  
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:(n_recommendations+1)]
    similar_games = [i[0] for i in sim_scores]
    
    print(similar_games)
    
    sim_games = []

    for game in games_df['app_name'].iloc[similar_games]:
        sim_games.append( game )

    return  title, sim_games


In [136]:
title, rec_games = recomend_similar_games( 'Football')
print("{}".format(title))
print("Recomendaciones para {}:".format(title))

for  game in rec_games:
    print("\t{}".format(game))

idx game =  1337
[41, 144, 198, 210, 211]
Lords of Football
Recomendaciones para Lords of Football:
	Orbitality
	Tank Universal
	Larva Mortus
	The Maw
	The Maw: River Redirect


In [137]:
title, rec_games = recomend_similar_games( 'Doom')
print("{}".format(title))
print("Recomendaciones para {}:".format(title))

for  game in rec_games:
    print("\t{}".format(game))

idx game =  84
[152, 156, 319, 369, 415]
Ultimate Doom
Recomendaciones para Ultimate Doom:
	S.T.A.L.K.E.R.: Clear Sky
	The Witcher: Enhanced Edition Director's Cut
	SpellForce - Platinum Edition
	Mount & Blade: Warband
	Alpha Protocol™


In [138]:
title, rec_games = recomend_similar_games( 'Tom Clancy')
print("{}".format(title))
print("Recomendaciones para {}:".format(title))

for  game in rec_games:
    print("\t{}".format(game))

idx game =  126
[98, 120, 121, 122, 123]
Tom Clancy's Splinter Cell®
Recomendaciones para Tom Clancy's Splinter Cell®:
	PuzzleQuest: Challenge of the Warlords
	Mystery P.I.™ - The Vegas Heist
	Luxor
	Reaxxion
	Little Farm


In [142]:
title, rec_games = recomend_similar_games( 'NBA')
print("{}".format(title))
print("Recomendaciones para {}:".format(title))

for  game in rec_games:
    print("\t{}".format(game))

idx game =  7297
[294, 493, 616, 1425, 1465]
NBA 2K17
Recomendaciones para NBA 2K17:
	Windosill
	Doc Clock: The Toasted Sandwich of Time
	Hamilton's Great Adventure
	Toki Tori 2+
	The Dream Machine: Chapter 4


### Verificaciones manuales sobre la similitud de los juegos

In [143]:
games_df.iloc[7297]

publisher                                                      GameHouse
genres                                        [Adventure, Casual, Indie]
app_name                             Delicious - Emily's Christmas Carol
title                                Delicious - Emily's Christmas Carol
release_date                                                  2016-11-04
tags                         [Casual, Adventure, Indie, Family Friendly]
discount_price                                                       NaN
specs                                [Single-player, Steam Achievements]
price                                                              12.99
early_access                                                       False
id                                                                544600
developer                                         Blue Giraffe,GameHouse
sentiment                                                  Very Positive
metascore                                          

In [125]:
games_df.iloc[1337]

publisher                                                        Digital Tribe
genres                                                         [Action, Indie]
app_name                                                              Bad Bots
title                                                                 Bad Bots
release_date                                                        2013-05-17
tags                         [Indie, Action, Platformer, Shooter, Metroidva...
discount_price                                                             NaN
specs                        [Single-player, Steam Achievements, Full contr...
price                                                                     4.99
early_access                                                             False
id                                                                      235070
developer                                                  Point Five Projects
sentiment                                           